## 기본설정 및 함수정의

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 아산시청 위도, 경도
Asan = [36.789882248764656, 127.00274491353838]

# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 아산시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [3]:
# shp to geodataframe convert
shapefile_path = "SBJ_2405_001/_census_data_2023_bnd_dong_bnd_dong_34040_2023_2023/bnd_dong_34040_2023_2023_2Q.shp"
asan_gdf = gpd.read_file(shapefile_path)
asan_gdf = asan_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 행정동 구분에 따라 색 구분하는 함수
def hjd_color(name):
    if name[-1] == '읍':
        return 'green'
    elif name[-1] == '면':
        return 'yellow'
    elif name[-1] == '동':
        return 'red'

asan_gdf['color'] = asan_gdf['ADM_NM'].apply(hjd_color)

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 필터링된 원도심 grid(격자) 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
for _, row in asan_gdf.iterrows() :
    color = row['color']
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature, color=color:{
            'fillColor': color,
            'color': 'black',
            'weight': 1
        }).add_to(m)
# 맵 출력
m

In [68]:
m.save("visualization/아산시 행정경계.html")

#### 격자(매핑용)

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/12.아산시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

In [75]:
grid_map_df

,type,properties.gid,geometry.type,geometry.coordinates,geometry
0,Feature,다바578741,Polygon,"[[[127.02655717320954, 36.86422355955024], [12...","POLYGON ((127.02656 36.86422, 127.02655 36.865..."
1,Feature,다바431650,Polygon,"[[[126.86232658848016, 36.781423593543536], [1...","POLYGON ((126.86233 36.78142, 126.86232 36.782..."
2,Feature,다바624804,Polygon,"[[[127.07785071414338, 36.92120830790351], [12...","POLYGON ((127.07785 36.92121, 127.07785 36.922..."
3,Feature,다바553584,Polygon,"[[[126.99943221243078, 36.722582478245315], [1...","POLYGON ((126.99943 36.72258, 126.99943 36.723..."
4,Feature,다바476680,Polygon,"[[[126.91254872882273, 36.80872603129526], [12...","POLYGON ((126.91255 36.80873, 126.91254 36.809..."
...,...,...,...,...,...
55029,Feature,다바468657,Polygon,"[[[126.90374117665121, 36.78794862412855], [12...","POLYGON ((126.90374 36.78795, 126.90373 36.788..."
55030,Feature,다바445733,Polygon,"[[[126.87740969585435, 36.856324288508205], [1...","POLYGON ((126.87741 36.85632, 126.87740 36.857..."
55031,Feature,다바619594,Polygon,"[[[127.07328965706101, 36.73188493870408], [12...","POLYGON ((127.07329 36.73188, 127.07328 36.732..."
55032,Feature,다바532640,Polygon,"[[[126.97557306589434, 36.77296209669358], [12...","POLYGON ((126.97557 36.77296, 126.97557 36.773..."


In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 필터링된 원도심 grid(격자) 추가

for _, row in asan_gdf.iterrows() :
    color = row['color']
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature, color=color:{
            'fillColor': color,
            'color': 'black',
            'weight': 1
        }).add_to(m)

folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = grid_map_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 0.3
    }
)
grid_layer.add_to(m)
# 맵 출력
m

In [74]:
m.save("visualization/아산시 격자.html")

#### 아산시 상세 도로망

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/4.아산시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_layer = folium.GeoJson(
    unary_union(asan_gdf['geometry']),
    name="아산시 행정동",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)
roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 맵 출력
m

In [78]:
m.save("visualization/아산시 도로망.html")

## 아산시 주민등록인구 현황

In [6]:
# 주민등록인구 data load
pop_df = pd.read_csv('SBJ_2405_001/2.아산시_주민등록인구현황.csv')
pop_df['year'] = pop_df['year'].astype(str)
pop_df['adm_nm'] = pop_df['adm_nm'].apply(lambda x:x.split(" ")[-1])
# 남성인구와 여성인구를 더하여 tot_pop(종합인구) column 생성
pop_df['tot_pop'] = pop_df.iloc[:,3:5].sum(axis=1)
pop_df

,adm_cd,adm_nm,year,m_pop,fm_pop,tot_pop
0,4420025000,염치읍,202101,3411,3106,6517
1,4420025300,배방읍,202101,39078,36929,76007
2,4420031000,송악면,202101,2174,2072,4246
3,4420033000,탕정면,202101,11783,9959,21742
4,4420035000,음봉면,202101,10815,9175,19990
...,...,...,...,...,...,...
607,4420058000,온양2동,202312,5390,4710,10100
608,4420059000,온양3동,202312,19547,19569,39116
609,4420060000,온양4동,202312,8590,8339,16929
610,4420061000,온양5동,202312,10603,10104,20707


In [7]:
# target값 list 생성 및 평균값 높은 target순으로 정렬
targets = []
for target in pop_df['adm_nm'].unique():
    pop_hjd = pop_df[pop_df['adm_nm'] == (target)]
    targets.append([target,pop_hjd['tot_pop'].mean()])
targets = sorted(targets, key=lambda x : x[1], reverse=True)
targets = [target[0] for target in targets]

# 행정동별 시계열 인구추이 dict생성
hjd_pop_dic = {}
for target in targets:
    pop_hjd = pop_df[pop_df['adm_nm'] == (target)]
    pop_hjd = pop_hjd[['year', 'm_pop', 'fm_pop', 'tot_pop']].reset_index(drop=True)
    pop_hjd.rename(columns = {'m_pop' : '%s_남'%target, 'fm_pop' : '%s_여'%target, 'tot_pop' : '%s_총합'%target}, inplace = True)
    hjd_pop_dic[target] = pop_hjd

In [ ]:
colors =px.colors.sequential.Plasma
colors = colors * 2
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 필터링된 원도심 grid(격자) 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
for i, row in asan_gdf.iterrows() :
    color = colors[i]
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature, color=color:{
            'fillColor': color,
            'color': 'black',
            'weight': 1
        }).add_to(m)
# 맵 출력
m

In [ ]:
# 빈 그래프
fig = go.Figure()

# for문을 통해 라인을 추가
for target in targets:
    fig.add_trace(go.Scatter(x=hjd_pop_dic[target]['year'], y=hjd_pop_dic[target]['%s_총합'%target],
                             mode='lines+markers',
                             name=target))
# 색상 업데이트
colors =px.colors.sequential.Plasma # 컬러맵
for i, trace in enumerate(fig.data):
    trace.line.color = colors[i % len(colors)]
    
# 그래프 타이틀 및 레이블 설정
fig.update_layout(
    title='<b>행정동별 시계열 인구변화</b>',
    xaxis_title='연도',
    yaxis_title='인구수',
    width=1000,
    height=600
)

# 그래프 표시
fig.show()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Subplot figure(4x5)
fig = make_subplots(
    rows=3, 
    cols=6, 
    subplot_titles=["<b>%s</b>" % target for target in targets],
    vertical_spacing=0.05,  # 세로 여백
    horizontal_spacing=0.05  # 가로 여백
)

# 17개의 타겟 값에 대해 반복하여 그래프 추가
for i in range(17):
    df = hjd_pop_dic[targets[i]]
    # 각 subplot에 trace 추가
    row = (i // 6) + 1
    col = (i % 6) + 1
    fig.add_trace(go.Scatter(x=df['year'], y=df['%s_총합'%targets[i]], mode='lines', name='%s'%targets[i]), row=row, col=col)
    fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        tickvals=['202106', '202206', '202306'],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
    fig.update_yaxes(
        showticklabels=True, 
        showgrid=True,
    )
# 색상 업데이트
colors =px.colors.sequential.Plasma # 컬러맵
for i, trace in enumerate(fig.data):
    trace.line.color = colors[i % len(colors)]
# 레이아웃 업데이트
fig.update_layout(height=1000, width=2200)

# 그래프 출력
fig.show()

In [ ]:
# Subplot figure(4x5)
fig = make_subplots(
    rows=3, 
    cols=6, 
    subplot_titles=["<b>%s</b>" % target for target in targets],
    vertical_spacing=0.05,  # 세로 여백
    horizontal_spacing=0.05  # 가로 여백
)

# 17개의 타겟 값에 대해 반복하여 그래프 추가
for i in range(17):
    df = hjd_pop_dic[targets[i]]
    # 각 subplot에 trace 추가
    row = (i // 6) + 1
    col = (i % 6) + 1
    fig.add_trace(go.Scatter(x=df['year'], y=df['%s_남' % targets[i]], mode='lines', name='남성', line=dict(color='blue')), row=row, col=col)
    fig.add_trace(go.Scatter(x=df['year'], y=df['%s_여' % targets[i]], mode='lines', name='여성', line=dict(color='red')), row=row, col=col)
    fig.update_xaxes(
        row=row,
        col=col,
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        tickvals=['202106', '202206', '202306'],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
    fig.update_yaxes(
        row=row,
        col=col,
        showticklabels=True, 
        showgrid=True,
    )

fig.update_layout(height=1000, width=2200)

# 그래프 출력
fig.show()


### 아산시 거주인구 필터링

국제연합(UN)의 기준에 따르면 전체 인구에서 65세 이상이 차지하는 비율인 고령자 인구 비율이 7% 이상이면 고령화 사회, 14% 이상이면 고령 사회, 20% 이상이면 초고령 사회로 구분된다.

국제 연합 기준(UN)과 국내 국민연금 수급연령(연금 지급개시연령) 및 근로자의 법상 정년(노동관계법령상 정년(60세) 의무규정)을 고려하여 60대를 기준으로 고령화 비율을 나누었다. 

In [8]:
respop_df = pd.read_csv('SBJ_2405_001/1.아산시_거주인구(격자).csv')

#아산시 grid ID 리스트 와 매핑 & respop_df 필터링
grid_id = grid_map_df['properties.gid'].tolist() # gid는 전부 unique한 값
grid_respop = respop_df[respop_df['gid'].isin(grid_id)]
grid_respop = grid_respop.fillna(0)
grid_respop = pd.concat([grid_respop.iloc[:, :2], grid_respop.iloc[:, 2:].astype(int)], axis=1)
grid_respop['year'] = grid_respop['year'].astype(str)

# 성별구분 없이 통합 및 세대별 인구수 데이터프레임으로 변환
columns = ['gid', 'year']
for i in range(2, len(grid_respop.columns)):
    age = f'{grid_respop.columns[i][2:4]}대' if grid_respop.columns[i][2] != '1' else f'{grid_respop.columns[i][2:5]}대'
    sex = '남' if grid_respop.columns[i][0] != 'm' else '여'
    columns.append('%s_%s'%(age, sex))
grid_respop.columns = columns
# 인구 column & 고령인구 column & 고령인구 비율(%) column 생성
grid_respop['인구'] = grid_respop.iloc[:, 2:].sum(axis=1)
grid_respop =  grid_respop[grid_respop['인구'] != 0] # 거주인구가 없는 격자 필터링(제외)
grid_respop['고령인구'] = grid_respop.iloc[:, 10:-1].sum(axis=1)
grid_respop['고령인구_비율'] = grid_respop.iloc[:, -1] / grid_respop.iloc[:, -2]
grid_respop = grid_respop.fillna(0) # NaN값 0으로 대체
grid_respop['고령인구_비율'] = grid_respop['고령인구_비율'].apply(lambda x : round(x*100, 2))

In [97]:
grid_respop

,gid,year,20대_여,20대_남,30대_여,30대_남,40대_여,40대_남,50대_여,50대_남,60대_여,60대_남,70대_여,70대_남,80대_여,80대_남,90대_여,90대_남,100대_여,100대_남,인구,고령인구,고령인구_비율
3,다바553584,2023,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0.00
16,다바568639,2023,21,7,8,11,17,18,24,26,11,9,0,0,0,0,0,0,0,0,152,20,13.16
26,다바562630,2023,24,12,26,19,32,30,38,42,25,22,9,0,0,6,0,0,0,0,285,62,21.75
55,다바561678,2023,16,8,8,7,20,16,20,21,12,9,0,11,0,0,0,0,0,0,148,32,21.62
68,다바554647,2023,0,0,6,0,0,0,9,8,17,17,10,0,0,10,0,0,0,0,77,54,70.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136572,다바559657,2021,8,9,8,8,7,7,9,9,21,18,0,9,0,0,0,0,0,0,113,48,42.48
138104,다바631647,2021,10,15,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38,0,0.00
141887,다바630647,2021,52,95,65,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269,0,0.00
155079,다바649644,2021,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,6,6,100.00


#### 아산시 거주인구 연도별 변화(line + marker graph)

In [9]:
year_lst = range(2021, 2024)
year_pop = [] # 인구수 종합
elder_pop = [] # 고령인구 종합
elder_rate = [] # 고령인구 비율
non_elder_pop = [] # 비고령인구 종합
non_elder_rate = [] # 비고령인구 비율
for year in year_lst:
    # 해당 year의 인구수 종합
    year_pop.append(grid_respop[grid_respop['year'] == str(year)]['인구'].sum())
    # 해당 year의 고령인구 종합
    elder_pop.append(grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum())
    # 해당 year의 고령인구 비율
    rate = grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum() / grid_respop[grid_respop['year'] == str(year)]['인구'].sum()
    elder_rate.append(round(rate * 100, 2))
    # 해당 year의 비고령인구 종합
    non_elder = grid_respop[grid_respop['year'] == str(year)]['인구'].sum() - grid_respop[grid_respop['year'] == str(year)]['고령인구'].sum()
    non_elder_pop.append(non_elder)
    # 해당 year의 고령인구 비율
    rate = non_elder / grid_respop[grid_respop['year'] == str(year)]['인구'].sum()
    non_elder_rate.append(round(rate * 100, 2))

pop_df = pd.DataFrame({'연도':year_lst, '종합인구':year_pop,'고령인구':elder_pop, '비고령인구':non_elder_pop, '고령인구 비율':elder_rate, '비고령인구 비율':non_elder_rate})
pop_df

,연도,종합인구,고령인구,비고령인구,고령인구 비율,비고령인구 비율
0,2021,188186,33452,154734,17.78,82.22
1,2022,185931,35347,150584,19.01,80.99
2,2023,182558,37357,145201,20.46,79.54


In [ ]:
fig = px.line(pop_df, x='연도', y='종합인구', labels={'x':'연도', 'y':'인구수'}, title='아산시 연도별 거주인구 추이')
fig.update_traces(mode='lines+markers')

# 각 마커 위에 주석을 추가합니다.
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['종합인구'][i] + 400,
        text=f"{pop_df['종합인구'][i]}명",
        showarrow=False,
    )
fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
fig.show()

In [ ]:
fig = px.line(pop_df, x='연도', y=['고령인구', '비고령인구'], labels={'x':'연도', 'y':'인구수'}, title='아산시 연도별 고령거주인구 vs 비고령거주인구 변화')
fig.update_traces(mode='lines+markers')

# 각 마커 위에 주석을 추가
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['고령인구'][i] + 7000,
        text=f"{pop_df['고령인구'][i]}명",
        showarrow=False,
    )

for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['비고령인구'][i] + 7000,
        text=f"{pop_df['비고령인구'][i]}명",
        showarrow=False,
    )

fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
# 범례 위치 설정 및 범례 이름 변경
fig.update_layout(
    legend=dict(x=1, y=1),
    legend_title_text='구분',
)
fig.show()

In [100]:
fig = px.line(pop_df, x='연도', y=['고령인구 비율', '비고령인구 비율'], labels={'x':'연도', 'y':'인구비율(%)'}, title='연도별 원도심 구역 고령인구 비율 변화')
fig.update_traces(mode='lines+markers')

# 각 마커 위에 주석을 추가
for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['고령인구 비율'][i] + 4,
        text=f"{pop_df['고령인구 비율'][i]}%",
        showarrow=False,
    )

for i in range(len(year_lst)):
    fig.add_annotation(
        x=pop_df['연도'][i],
        y=pop_df['비고령인구 비율'][i] + 4,
        text=f"{pop_df['비고령인구 비율'][i]}%",
        showarrow=False,
    )

fig.update_xaxes(
        showticklabels=True, 
        showgrid=False, 
        tickmode='array',
        range=[2020.5, 2023.5],
        tickvals=[2021, 2022, 2023],  # x축에 표시할 값
        ticktext=['2021', '2022', '2023'],  # x축에 표시할 텍스트
    )
fig.update_yaxes(
    showticklabels=True, 
    showgrid=True,
    )
# 범례 위치 설정 및 범례 이름 변경
fig.update_layout(
    legend=dict(x=1, y=1),
    legend_title_text='구분',
)
fig.show()

#### 아산시 거주인구 시각화(격자매핑)

In [10]:
# 격자 ID와 격자 폴리곤 각각을 매핑한 딕셔너리 생성
map_dic = {}
for _, row in grid_map_df.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

# 생성된 map_dic에 filtered_respop의 gid를 매핑하여 'geometry' column 생성
geometry_lst = []
for _, row in grid_respop.iterrows():
    geometry_lst.append(map_dic[row['gid']])
grid_respop['geometry'] = geometry_lst

In [ ]:
# color 컬럼 생성 및 조건에 따라 값 설정
def get_color(population, q75, q50, q25):
    if population > q75:
        return 'red'
    elif population > q50:
        return 'orange'
    elif population > q25:
        return 'yellow'
    else:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 350px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 거주인구 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 거주인구 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 거주인구 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 거주인구 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 거주인구 상위 100%<br>
     &nbsp; <i style="background:white">&nbsp;</i>&nbsp; 거주민 없는 구역
     </div>
     """

# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('cartodbpositron').add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 연도별 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)
m_2023 = folium.FeatureGroup(name="2023년", overlay=False)

# 연도와 레이어를 매핑하려 dict 형식으로 저장
year_lst = [str(year) for year in range(2021, 2024)]
layers_lst = [m_2021, m_2022, m_2023]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 그 연도에 해당하는 레이어에 추가
for year in year_lst:  # 2021부터 2023까지의 연도
    filtered_grid_respop_year = grid_respop[grid_respop['year'] == year]
    max_v = filtered_grid_respop_year['인구'].max()
    # 인구의 상위 25%, 50%, 75%에 해당하는 값 계산
    q25 = filtered_grid_respop_year['인구'].quantile(0.25)
    q50 = filtered_grid_respop_year['인구'].quantile(0.50)
    q75 = filtered_grid_respop_year['인구'].quantile(0.75)
    filtered_grid_respop_year['color'] = filtered_grid_respop_year['인구'].apply(lambda x : get_color(x, q75, q50, q25))

    # 'gid' 값을 기준으로 그룹화
    for gid in tqdm(filtered_grid_respop_year['gid'].unique()):
        subset = filtered_grid_respop_year[filtered_grid_respop_year['gid'] == gid].copy()
        subset_pop = subset['인구'].iloc[0]
        color = subset['color'].iloc[0]
        folium.GeoJson(
            filtered_grid_respop_year[filtered_grid_respop_year['gid'] == gid]['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year])
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [127]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 거주인구 시각화(격자매핑).html")

#### 아산시 거주인구 HEATMAP - 연도선택

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 연도별 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)
m_2023 = folium.FeatureGroup(name="2023년", overlay=False)

# 연도와 레이어를 매핑하려 dict 형식으로 저장
year_lst = [str(year) for year in range(2021, 2024)]
layers_lst = [m_2021, m_2022, m_2023]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2021부터 2023까지의 연도
    subset = grid_respop[grid_respop['year'] == year]
    # '인구'과 'geometry'을 사용하여 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], aging] for point, aging in zip(subset['geometry'], subset['인구'])]
    HeatMap(heat_data, max_opacity=0.6, radius=20).add_to(layers_dic[year])

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [134]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 거주인구 HEATMAP.html")

#### 아산시 거주인구 고령화 정도 시각화(격자매핑)

In [ ]:
#  고령화 정도에 따라 grid색상을 지정하는 함수
def rate_color(rate):
    if rate > 20:
        return 'RGBA(255,0,1)' # 빨간색
    elif 20 > rate >= 14:
        return 'RGBA(255,165,1)' # 주황색
    elif 14 > rate >= 7:
        return 'RGBA(255,255,1)' # 노란색
    elif 7 > rate >= 0:
        return 'RGBA(0,128,1)' # 초록색

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 350px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 고령화 정도 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 초고령 구역 (60세 이상인구 20% 이상)<br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 고령 구역 (60세 이상인구 14% 이상 20% 이하)<br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 고령화 구역 (60세 이상인구 7% 이상 14% 이하)<br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 일반 구역 (60세 이상인구 7% 이하)<br>
     &nbsp; <i style="background:white">&nbsp;</i>&nbsp; 거주민 없는 구역
     </div>
     """

# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('cartodbpositron').add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 연도별 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)
m_2023 = folium.FeatureGroup(name="2023년", overlay=False)

# 연도와 레이어를 매핑하려 dict 형식으로 저장
year_lst = [str(year) for year in range(2021, 2024)]
layers_lst = [m_2021, m_2022, m_2023]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 그 연도에 해당하는 레이어에 추가
for year in year_lst:  # 2021부터 2023까지의 연도
    filtered_grid_respop_year = grid_respop[grid_respop['year'] == year]

    # 'gid' 값을 기준으로 그룹화
    for gid in tqdm(filtered_grid_respop_year['gid'].unique()):
        subset = filtered_grid_respop_year[filtered_grid_respop_year['gid'] == gid].copy()
        rate = subset['고령인구_비율'].iloc[0]
        color = rate_color(rate)
        subset['고령인구_비율'] = subset['고령인구_비율'].apply(lambda x: str(x) + '%')

        # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장(격자를 클릭하면 df 팝업됨)
        subset_popup = subset.iloc[:, 1:-1].style.hide(axis='index').set_properties(
            **{'border': '1px solid black'}).to_html()  # [['연도', '고령인구_비율']]
        popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
        color = rate_color(rate)
        folium.GeoJson(
            filtered_grid_respop_year[filtered_grid_respop_year['gid'] == gid]['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=1200), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [140]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 고령화정도 시각화(격자매핑).html")

#### 아산시 거주인구 고령화 정도 HEATMAP - 연도선택 가능

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 연도별 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)
m_2023 = folium.FeatureGroup(name="2023년", overlay=False)

# 연도와 레이어를 매핑하려 dict 형식으로 저장
year_lst = [str(year) for year in range(2021, 2024)]
layers_lst = [m_2021, m_2022, m_2023]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2021부터 2023까지의 연도
    subset = grid_respop[grid_respop['year'] == year]
    # '인구'과 'geometry'을 사용하여 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], aging] for point, aging in zip(subset['geometry'], subset['고령인구_비율'])]
    HeatMap(heat_data, max_opacity=0.6, radius=20).add_to(layers_dic[year])
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [144]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 고령화정도 HEATMAP.html")

#### 아산시 연도별 유동인구

In [147]:
# 유동인구 df 로드
floating_population_df = pd.read_csv('SBJ_2405_001/25.아산시_성연령별_유동인구.csv')
floating_population_df['std_ym'] = floating_population_df['std_ym'].astype(str)
floating_population_df = geo_transform(floating_population_df)
floating_population_df['pop'] = floating_population_df.iloc[:, 1:-3].sum(axis=1).copy()
col_lst = list(floating_population_df.columns[:1]) + list(floating_population_df.columns[-1:]) + list(floating_population_df.columns[-2:-1])
floating_population_df = floating_population_df[col_lst]
floating_population_df['std_ym'] = floating_population_df['std_ym'].apply(lambda x:x[:4])

# 유동인구 정규화
max_val = floating_population_df['pop'].max()
floating_population_df['pop'] = floating_population_df['pop'].apply(lambda x : round((x/max_val)*100,2))

floating_population_dic = {}
for year in year_lst:
    floating_population_dic[year] = floating_population_df[floating_population_df['std_ym'] == year]

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 연도별 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)
m_2023 = folium.FeatureGroup(name="2023년", overlay=False)

# 연도와 레이어를 매핑하려 dict 형식으로 저장
year_lst = [str(year) for year in range(2021, 2024)]
layers_lst = [m_2021, m_2022, m_2023]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:
    # 'geometry' 열의 값을 위치 정보로 사용하여 유동인구 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], floating] for point, floating in tqdm(zip(floating_population_dic[year]['geometry'], floating_population_dic[year]['pop']))]
    HeatMap(heat_data, max_opacity=0.6, radius=15).add_to(layers_dic[year])
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [153]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 연도별 유동인구 HEATMAP.html")

#### 아산시 시간대별 유동인구

In [23]:
# 유동인구 df 로드
floating_population_df_T = pd.read_csv('SBJ_2405_001/26.아산시_시간대별_유동인구.csv')
floating_population_df_T['std_ym'] = floating_population_df_T['std_ym'].astype(str)
floating_population_df_T = geo_transform(floating_population_df_T)
floating_population_df_T['std_ym'] = floating_population_df_T['std_ym'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_T.columns[1:-3] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    numeric_part = column_name.split('_')[-1] # 숫자 부분 추출
    new_column_name = f"{numeric_part}시" # '시'를 붙여 새로운 열 이름 생성
    new_column_names.append(new_column_name) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_T.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)

# 연도별 유동인구 분할
floating_population_T_dic = {}
for year in year_lst:
    # geometry 컬럼을 문자열로 변환하여 그룹화
    gdf = floating_population_df_T[floating_population_df_T['std_ym'] == year]
    gdf['geometry_str'] = gdf['geometry'].astype(str)
    # 제외할 컬럼 리스트
    exclude_columns = ['std_ym', 'lon', 'lat', 'geometry', 'geometry_str']
    # 그룹화할 컬럼 리스트 (제외할 컬럼을 제외한 나머지)
    group_columns = [col for col in gdf.columns if col not in exclude_columns]
    # geometry_str 컬럼을 기준으로 그룹화하고, 제외한 컬럼을 제외한 나머지 컬럼의 평균을 계산
    grouped = gdf.groupby('geometry_str')[group_columns].mean()
    # geometry_str 컬럼을 제거하고 원래 geometry 컬럼을 복원
    grouped['geometry'] = gdf.groupby('geometry_str')['geometry'].first().values
    # 결과를 GeoDataFrame으로 변환
    grouped = gpd.GeoDataFrame(grouped, geometry='geometry')
    grouped.reset_index(drop=True, inplace=True)
    floating_population_T_dic[year] = grouped

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

##### 2021년 시간대별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_T_dic['2021']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for i in tqdm(range(24)):
    time_column = f'{i:02}시'
    volume = df_year[time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=5, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [156]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 시간대별 유동인구 2021 HEATMAP.html")

##### 2022년 시간대별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_T_dic['2022']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for i in tqdm(range(24)):
    time_column = f'{i:02}시'
    volume = df_year[time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=5, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [158]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 시간대별 유동인구 2022 HEATMAP.html")

##### 2023년 시간대별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_T_dic['2023']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for i in tqdm(range(24)):
    time_column = f'{i:02}시'
    volume = df_year[time_column]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[f'{i:02}시' for i in range(24)], auto_play=False, max_opacity=0.6, radius=5, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [160]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 시간대별 유동인구 2023 HEATMAP.html")

#### 아산시 요일별 유동인구

In [10]:
floating_population_df_D = pd.read_csv('SBJ_2405_001/27.아산시_요일별_유동인구.csv')

floating_population_df_D['std_ym'] = floating_population_df_D['std_ym'].astype(str)
floating_population_df_D = geo_transform(floating_population_df_D)
floating_population_df_D['std_ym'] = floating_population_df_D['std_ym'].apply(lambda x:x[:4])

# 열 이름 변경 과정
columns_to_rename = floating_population_df_D.columns[1:-3] # 시계열 정보를 포함한 column만 선택
new_column_names = []
# 각 열 이름을 처리하여 새로운 열 이름을 생성
for column_name in columns_to_rename:
    day_part = column_name.split('_')[0] # 숫자 부분 추출
    new_column_names.append(day_part) # new_column_names에 추가
# 열 이름을 변경합니다.
floating_population_df_D.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)

# 연도별 유동인구 분할
floating_population_D_dic = {}
for year in year_lst:
    # geometry 컬럼을 문자열로 변환하여 그룹화
    gdf = floating_population_df_D[floating_population_df_D['std_ym'] == year]
    gdf['geometry_str'] = gdf['geometry'].astype(str)
    # 제외할 컬럼 리스트
    exclude_columns = ['std_ym', 'lon', 'lat', 'geometry', 'geometry_str']
    # 그룹화할 컬럼 리스트 (제외할 컬럼을 제외한 나머지)
    group_columns = [col for col in gdf.columns if col not in exclude_columns]
    # geometry_str 컬럼을 기준으로 그룹화하고, 제외한 컬럼을 제외한 나머지 컬럼의 평균을 계산
    grouped = gdf.groupby('geometry_str')[group_columns].mean()
    # geometry_str 컬럼을 제거하고 원래 geometry 컬럼을 복원
    grouped['geometry'] = gdf.groupby('geometry_str')['geometry'].first().values
    # 결과를 GeoDataFrame으로 변환
    grouped = gpd.GeoDataFrame(grouped, geometry='geometry')
    grouped.reset_index(drop=True, inplace=True)
    floating_population_D_dic[year] = grouped

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

##### 2021년 요일별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_D_dic['2021']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for day in tqdm(df_year.columns[:7]):
    volume = df_year[day]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in df_year.columns[:7]], auto_play=False, max_opacity=0.6, radius=3, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [172]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 요일별 유동인구 2021 HEATMAP.html")

##### 2022년 요일별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_D_dic['2022']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for day in tqdm(df_year.columns[:7]):
    volume = df_year[day]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in df_year.columns[:7]], auto_play=False, max_opacity=0.6, radius=3, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [174]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 요일별 유동인구 2022 HEATMAP.html")

##### 2023년 요일별 유동인구 변화

In [ ]:
# 해당년도 df불러오기
df_year = floating_population_D_dic['2023']

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 기본 배경지도를 항상 표시하도록 설정
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = df_year['geometry'].y
longitudes = df_year['geometry'].x
# 시간대별 히트맵 데이터 생성
heat_data = []
for day in tqdm(df_year.columns[:7]):
    volume = df_year[day]
    heat_data_hour = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_hour)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in df_year.columns[:7]], auto_play=False, max_opacity=0.6, radius=3, name='유동인구HEATMAP').add_to(m)


# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [12]:
m.save("visualization/1. 아산시 인구변화 분석/아산시 요일별 유동인구 2023 HEATMAP.html")